In [67]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from scipy import ndimage  

In [68]:
# Define the base ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)

In [69]:
# Define the number of classes in your dataset
num_classes = 2

# Add your custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [70]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [71]:
# Freeze the layers of the ResNet50 base model
for layer in base_model.layers:
    layer.trainable = False

In [72]:
# Compile the model
model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])


In [73]:
# Set up data generators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

batch_size=16

In [74]:
train_generator = train_datagen.flow_from_directory(
    './dummy_dataset/train',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 350 images belonging to 2 classes.


In [75]:
validation_generator = test_datagen.flow_from_directory(
    './dummy_dataset/valid',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 235 images belonging to 2 classes.


In [76]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=7,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/7
21/21 [==============================] - 29s 1s/step - loss: 0.0651 - accuracy: 0.9581 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/7
21/21 [==============================] - 27s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/7
21/21 [==============================] - 26s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/7
21/21 [==============================] - 25s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/7
21/21 [==============================] - 25s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/7
21/21 [==============================] - 24s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/7
21/21 [==============================] - 25s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+0

In [77]:
# Save the trained model
model.save('./models/Resnet50.h5')

e:\vscode\Weed Detection\ML\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [78]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import decode_predictions
import numpy as np
import os

In [79]:
# Load your trained model 
model = load_model('./models/Resnet50.h5')

# Path to the folder containing test images
test_folder = './dummy_dataset/test/images'

In [80]:
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [81]:
def predict_with_resnet50(model, img_path):
    x = load_and_preprocess_image(img_path)
    preds = model.predict(x)
    return preds

In [82]:
for filename in os.listdir(test_folder):
   
    img_path = os.path.join(test_folder, filename)
    predictions = predict_with_resnet50(model, img_path)
        
    # Get the index with the maximum probability
    predicted_class_index = np.argmax(predictions)

    print(f"Image: {filename}, Predicted class index: {predicted_class_index}")

1/1 [==============================] - 1s 1s/step
Image: 12122_jpg.rf.d7313af9bab0e80b7149ee0a3c32caf8.jpg, Predicted class index: 0
1/1 [==============================] - 0s 89ms/step
Image: 12198_jpg.rf.a216dcc7b89a168893ccd130a89553ee.jpg, Predicted class index: 0
1/1 [==============================] - 0s 103ms/step
Image: 12205_jpg.rf.ab3db8b3b513ffeae1e66458ae3b1df9.jpg, Predicted class index: 0
1/1 [==============================] - 0s 87ms/step
Image: 301_jpg.rf.e0a19ebd17c5738934d155d4747cecba.jpg, Predicted class index: 0
1/1 [==============================] - 0s 88ms/step
Image: 310_jpg.rf.a94f591dac795f5127eb44adb9887dae.jpg, Predicted class index: 0
1/1 [==============================] - 0s 103ms/step
Image: 311_jpg.rf.c0ec2f814a7f68cab8d1c06ef9ec24a2.jpg, Predicted class index: 0
1/1 [==============================] - 0s 83ms/step
Image: 317_jpg.rf.c7f0907204ab5da50ae42cf2336b2444.jpg, Predicted class index: 0
1/1 [==============================] - 0s 85ms/step
Image: 3217